# Database Migrations with Alembic in FastAPI

In this tutorial, we'll explore how to manage database migrations in a FastAPI application using **Alembic**. We'll start by understanding what Alembic is, delve into its key components, and then walk through setting up and using Alembic step by step. By the end of this tutorial, you'll have a solid understanding of how to integrate Alembic into your FastAPI projects for efficient database schema management.

## Table of Contents

1. [Introduction to Alembic](#1-introduction-to-alembic)
2. [Understanding Alembic's Key Components](#2-understanding-alembics-key-components)
3. [Prerequisites](#3-prerequisites)
4. [Setting Up the Environment](#4-setting-up-the-environment)
5. [Installing Dependencies](#5-installing-dependencies)
6. [Creating the FastAPI Application](#6-creating-the-fastapi-application)
7. [Setting Up the Database](#7-setting-up-the-database)
8. [Integrating SQLAlchemy with FastAPI](#8-integrating-sqlalchemy-with-fastapi)
9. [Initializing Alembic](#9-initializing-alembic)
10. [Configuring Alembic](#10-configuring-alembic)
11. [Creating and Applying Migrations](#11-creating-and-applying-migrations)
12. [Working with Data Migrations](#12-working-with-data-migrations)
13. [Rolling Back Migrations](#13-rolling-back-migrations)
14. [Conclusion](#14-conclusion)
15. [References](#15-references)

## 1. Introduction to Alembic

**Alembic** is a lightweight database migration tool for SQLAlchemy. It helps you manage changes to your database schema over time, tracking versions and allowing you to upgrade or downgrade your database schema as needed.

### Why Use Alembic?

- **Version Control**: Keeps track of schema changes, similar to how Git tracks code changes.
- **Collaboration**: Allows multiple developers to work on the database schema without conflicts.
- **Automation**: Facilitates automated migrations in deployment pipelines.
- **Safety**: Provides rollback mechanisms in case of issues.

## 2. Understanding Alembic's Key Components

Before diving into implementation, let's familiarize ourselves with Alembic's main components.

### 2.1. Alembic Environment

- **`alembic.ini`**: Main configuration file for Alembic.
- **Migration Scripts Directory**: Contains versioned migration scripts.
  - **`env.py`**: Sets up the environment for migrations.

### 2.2. Migration Scripts

- **Revision Scripts**: Python files that define database schema changes.
  - **`upgrade()`**: Contains code to apply the migration.
  - **`downgrade()`**: Contains code to reverse the migration.

### 2.3. Alembic Commands

- **`alembic init`**: Initializes a new Alembic environment.
- **`alembic revision`**: Creates a new migration script.
- **`alembic upgrade`**: Applies migrations to the database.
- **`alembic downgrade`**: Reverts migrations.

## 3. Prerequisites

Ensure you have the following installed:

- **Python 3.7+**
- **FastAPI**
- **SQLAlchemy**
- **Alembic**
- **A SQL Database** (e.g., SQLite, PostgreSQL)
- **Basic Knowledge**: Familiarity with Python, FastAPI, SQLAlchemy, and database concepts.

## 4. Setting Up the Environment

Create a project directory and set up a virtual environment.

```bash
# Create project directory
mkdir fastapi-alembic-demo
cd fastapi-alembic-demo

# Create and activate virtual environment
python -m venv venv
source venv/bin/activate  # On Windows use: venv\Scripts\activate
```

## 5. Installing Dependencies

Install the necessary packages.

```bash
pip install fastapi uvicorn sqlalchemy alembic
```

- **`fastapi`**: Web framework for building APIs.
- **`uvicorn`**: ASGI server to run FastAPI apps.
- **`sqlalchemy`**: ORM for database interactions.
- **`alembic`**: Database migration tool.

## 6. Creating the FastAPI Application

Set up the basic structure of the application.

**Directory Structure**

```
fastapi-alembic-demo/
├── app/
│   ├── __init__.py
│   ├── main.py
│   ├── models.py
│   ├── schemas.py
│   └── database.py
```

Create the `app` directory and files.

```bash
mkdir app
touch app/__init__.py app/main.py app/models.py app/schemas.py app/database.py
```

## 7. Setting Up the Database

### 7.1. Configuring the Database URL

For simplicity, we'll use SQLite in this tutorial.

**app/database.py**

```python
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./test.db"  # SQLite URL

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)  # Needed for SQLite

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()
```

### 7.2. Dependency for Database Session

```python
from sqlalchemy.orm import Session

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
```

## 8. Integrating SQLAlchemy with FastAPI

### 8.1. Defining Models

Create a `User` model.

**app/models.py**

```python
from sqlalchemy import Column, Integer, String
from .database import Base

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, index=True)
    email = Column(String, unique=True, index=True)
```

### 8.2. Creating Pydantic Schemas

**app/schemas.py**

```python
from pydantic import BaseModel

class UserBase(BaseModel):
    username: str
    email: str

class UserCreate(UserBase):
    pass

class User(UserBase):
    id: int

    class Config:
        orm_mode = True
```

### 8.3. Writing CRUD Operations

**app/main.py**

```python
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from . import models, schemas, database

app = FastAPI()

# Create tables
models.Base.metadata.create_all(bind=database.engine)

# Dependency
get_db = database.get_db

@app.post("/users/", response_model=schemas.User)
def create_user(user: schemas.UserCreate, db: Session = Depends(get_db)):
    db_user = models.User(username=user.username, email=user.email)
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user

@app.get("/users/{user_id}", response_model=schemas.User)
def read_user(user_id: int, db: Session = Depends(get_db)):
    db_user = db.query(models.User).filter(models.User.id == user_id).first()
    if db_user is None:
        raise HTTPException(status_code=404, detail="User not found")
    return db_user
```

## 9. Initializing Alembic

Initialize Alembic in your project.

```bash
alembic init alembic
```

This creates an `alembic` directory and an `alembic.ini` file.

## 10. Configuring Alembic

### 10.1. Editing `alembic.ini`

In `alembic.ini`, set the database URL.

```ini
[alembic]
sqlalchemy.url = sqlite:///./test.db
```

Alternatively, you can leave it empty and set it in `env.py`.

### 10.2. Modifying `env.py`

Update `env.py` to import your models and set `target_metadata`.

**alembic/env.py**

```python
import sys
import os
from logging.config import fileConfig

from sqlalchemy import engine_from_config
from sqlalchemy import pool
from alembic import context

# Add your app's directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from app.models import Base  # Import your models' Base
from app.database import SQLALCHEMY_DATABASE_URL  # Import the database URL

config = context.config

# Interpret the config file for Python logging.
fileConfig(config.config_file_name)

# Set the database URL
config.set_main_option('sqlalchemy.url', SQLALCHEMY_DATABASE_URL)

# Set the target metadata
target_metadata = Base.metadata

def run_migrations_offline():
    ...

def run_migrations_online():
    ...

if context.is_offline_mode():
    run_migrations_offline()
else:
    run_migrations_online()
```

**Note**: Ensure `target_metadata` is set to `Base.metadata` so Alembic knows about your models.

## 11. Creating and Applying Migrations

### 11.1. Generating an Initial Migration

Create a new revision.

```bash
alembic revision --autogenerate -m "Initial migration"
```

This generates a migration script in `alembic/versions/`.

### 11.2. Reviewing the Migration Script

Open the generated migration script to review the changes.

**alembic/versions/<revision_id>_initial_migration.py**

```python
def upgrade():
    # ### commands auto generated by Alembic - please adjust! ###
    op.create_table('users',
    sa.Column('id', sa.Integer(), nullable=False),
    sa.Column('username', sa.String(), nullable=True),
    sa.Column('email', sa.String(), nullable=True),
    sa.PrimaryKeyConstraint('id')
    )
    op.create_index(op.f('ix_users_email'), 'users', ['email'], unique=True)
    op.create_index(op.f('ix_users_id'), 'users', ['id'], unique=False)
    op.create_index(op.f('ix_users_username'), 'users', ['username'], unique=False)
    # ### end Alembic commands ###

def downgrade():
    # ### commands auto generated by Alembic - please adjust! ###
    op.drop_index(op.f('ix_users_username'), table_name='users')
    op.drop_index(op.f('ix_users_id'), table_name='users')
    op.drop_index(op.f('ix_users_email'), table_name='users')
    op.drop_table('users')
    # ### end Alembic commands ###
```

### 11.3. Applying the Migration

Apply the migration to the database.

```bash
alembic upgrade head
```

### 11.4. Verifying the Migration

Check that the `users` table has been created in your database.

## 12. Working with Data Migrations

Suppose you need to add a new `full_name` column to the `User` model.

### 12.1. Updating the Model

**app/models.py**

```python
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, index=True)
    email = Column(String, unique=True, index=True)
    full_name = Column(String, nullable=True)  # New column
```

### 12.2. Generating a New Migration

Create a new migration script.

```bash
alembic revision --autogenerate -m "Add full_name to users"
```

### 12.3. Reviewing the Migration Script

**alembic/versions/<revision_id>_add_full_name_to_users.py**

```python
def upgrade():
    op.add_column('users', sa.Column('full_name', sa.String(), nullable=True))

def downgrade():
    op.drop_column('users', 'full_name')
```

### 12.4. Applying the Migration

```bash
alembic upgrade head
```

### 12.5. Updating Schemas and Endpoints

**app/schemas.py**

```python
class UserBase(BaseModel):
    username: str
    email: str
    full_name: Optional[str] = None  # Updated schema

class UserCreate(UserBase):
    pass

class User(UserBase):
    id: int

    class Config:
        orm_mode = True
```

**app/main.py**

```python
@app.post("/users/", response_model=schemas.User)
def create_user(user: schemas.UserCreate, db: Session = Depends(get_db)):
    db_user = models.User(
        username=user.username,
        email=user.email,
        full_name=user.full_name  # Include new field
    )
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user
```

## 13. Rolling Back Migrations

If you need to revert the last migration:

```bash
alembic downgrade -1
```

This will execute the `downgrade()` function of the last migration script.

### 13.1. Downgrading to a Specific Revision

List the migration history:

```bash
alembic history
```

Downgrade to a specific revision ID:

```bash
alembic downgrade <revision_id>
```

## 14. Conclusion

In this tutorial, we've:

- Set up a FastAPI application with SQLAlchemy models.
- Initialized Alembic for database migrations.
- Configured Alembic to recognize our models.
- Created and applied migration scripts.
- Performed schema changes and data migrations.
- Rolled back migrations when necessary.

Alembic is a powerful tool that integrates seamlessly with SQLAlchemy, making database schema management more efficient and reliable.

## 15. References

- [Alembic Official Documentation](https://alembic.sqlalchemy.org/en/latest/)
- [FastAPI with SQL Databases](https://fastapi.tiangolo.com/tutorial/sql-databases/)
- [SQLAlchemy Official Documentation](https://docs.sqlalchemy.org/en/14/)
- [Pydantic Official Documentation](https://pydantic-docs.helpmanual.io/)